In [3]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import gc
sns.set_style("whitegrid");
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pickle as pkl
import tqdm as tqdm
from random import choices
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.linear_model import Ridge
import warnings
warnings.filterwarnings("ignore")

C:\Users\Lenovo\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
def one_hot_encoding_dataframe(df):
    '''
    one hot encoding 
    '''
    original_columns = list(df.columns)
    cat_columns=[x for x in df.columns if df[x].dtype == 'object']
    df=pd.get_dummies(df,columns=cat_columns,dummy_na= False)
    new_added_columns=list(set(df.columns).difference(set(original_columns)))
    return df,new_added_columns

In [7]:
def feature_engineering_on_app_train_test(test_point):
    
    '''
    final feature engineering applied on app_train and app_test
    
    '''
#     app_train=pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv')
#     app_test=pd.read_csv('/kaggle/input/home-credit-default-risk/application_test.csv')
#     #we merge the two data frames the preprocessing we on train must also be done on test
#     df=app_train.append(app_test).reset_index()
    
#     del app_train
#     del app_test
    
#     df=df[df['CODE_GENDER']!='XNA'] 

    df=test_point
    
#     print(df.head())
    
#     df["CODE_GENDER"].replace({'XNA': np.nan}, inplace = True)
    
#     df["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)
    
    
    with open("all_columns_app_train_test.pkl", "rb") as f:
        all_columns = pkl.load(f)
    
#     print(df.head())
    
    
#     with open("all_columns_app_train_test.pkl", "wb") as f:
#         pkl.dump(all_columns, f)
    
#     df=df[df['SK_ID_CURR']==id_value]

#     #The XNA value doesn't mean any thing so it is removed from train data
#     df=df[df['CODE_GENDER']!='XNA'] 
        
#     #we remove this because 365243 is an outlier


#     #There is an outlier in the train data where AMT_INCOME_TOTAL of a person having highest income had difficulty in paying loan. 
#     df=df[df['AMT_INCOME_TOTAL']<(0.2*1e8)]
 
#     print(list(df.columns))
    
    df['DOCUMNNET_COUNT']=(df[['FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3',
       'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6',
       'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9',
       'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12',
       'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15',
       'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18',
       'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21']]==1).sum(axis=1)
    
    
    df['AMT_REQ_CREDIT_BUREAU_HDWMQY']=(df[['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY',
       'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR']]).sum(axis=1)
    
    
    #Using domain knowledge and Pairplot
    
#     df['AMT_CREDIT_FLAG']=( df['AMT_CREDIT']<=300000).astype(int)
    
#     df['DAYS_BIRTH_FLAG']=(abs(df['DAYS_BIRTH']//365)<=19).astype(int)
    
    #pecentage of his life spent working
    df['DAYS_WORKING_PER']=df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    
    df['DAYS_UNEMPLOYED']=abs(df['DAYS_BIRTH'])-abs(df['DAYS_EMPLOYED'])
    
    df['GOODS_PRICE_INCOME_TOTAL_PER']=df['AMT_INCOME_TOTAL']/df['AMT_GOODS_PRICE']
    
    df['GOODS_PRICE_CREDIT_PER']=df['AMT_CREDIT']/df['AMT_GOODS_PRICE']
    
    df['GOODS_PRICE_AMT_ANNUITY_PER']=df['AMT_ANNUITY']/df['AMT_GOODS_PRICE']
    
#     df['GOODS_PRICE_EMP']=abs(df['DAYS_EMPLOYED'])/df['AMT_GOODS_PRICE']
    
#     df['AMT_CREDIT_BIRTH']=df['AMT_CREDIT']/abs(df['DAYS_BIRTH']/365)
    
    #percentage income of person and the credit amount
    df['INCOME_CREDIT_PER'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']

    #percentage income of person and the credit amount
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / (df['CNT_FAM_MEMBERS']+1)

    #Amount paid for previous loan application every month decided by the number of day employed
    df['ANNUITY_DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED']/ df['AMT_ANNUITY']
    
    df['AMT_CREDIT_DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED']/ df['AMT_CREDIT']
    
    #Amount paid for  loan application every month decided by the number of day lived
    df['ANNUITY_DAYS_BIRTH_PERC'] = df['DAYS_BIRTH']/ df['AMT_ANNUITY']

    #Anually paid amount to amount credited
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    
    df['PAYMENT_RATE_INV'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']

    df['PAY_TOWARDS_LOAN'] = df['AMT_INCOME_TOTAL']-df['AMT_ANNUITY']

    # df['AMT_INCOME_TOTAL_FLAG_LOAN_LESS_50'] =(df['AMT_ANNUITY']<=(0.50*df['AMT_INCOME_TOTAL'])).astype(int)
    
    df['MEAN_DEFAULT_SURR']=((df[['OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE']]).sum(axis=1))//4
    
    df['ADDRESS_MISSMATCH']=((df[['REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
       'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY','LIVE_CITY_NOT_WORK_CITY']]).sum(axis=1))
    
    df['MEAN_ENQUIRIES']=((df[['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY',
       'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR']]).mean(axis=1))
    
    df['CONTACT_REF']=((df[['FLAG_MOBIL', 'FLAG_EMP_PHONE',
       'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE','FLAG_PHONE','FLAG_EMAIL']]).sum(axis=1))
    
    df['MAX_DAYS_SOMETHING_CHANGED']=((df[['DAYS_EMPLOYED', 'DAYS_ID_PUBLISH',
       'DAYS_REGISTRATION']]).max(axis=1))
    
    #Creating features from useful features
    df['EXT_SOURCE_MEAN']=(df[['EXT_SOURCE_1', 'EXT_SOURCE_2',
       'EXT_SOURCE_3']]).mean(axis=1)
    
    df['EXT_SOURCE_MEDIAN']=(df[['EXT_SOURCE_1', 'EXT_SOURCE_2',
       'EXT_SOURCE_3']]).median(axis=1)
    
    df['EXT_SOURCE_MIN']=(df[['EXT_SOURCE_1', 'EXT_SOURCE_2',
       'EXT_SOURCE_3']]).min(axis=1)
    
    df['EXT_SOURCE_MAX']=(df[['EXT_SOURCE_1', 'EXT_SOURCE_2',
       'EXT_SOURCE_3']]).max(axis=1)
    
    
    df,new_cat_columns=one_hot_encoding_dataframe(df)
    add_columns=list(set(all_columns)-set(df.columns))
    for col in add_columns:
        df[col]=0
    df['TARGET']=np.nan    
    return df

In [9]:
def fill_the_missing_values(df):
    
    with open("missing_3_list_7.pkl", "rb") as f:
        missing_3_list = pkl.load(f)
    with open("missing_2_list (1).pkl", "rb") as f:
        missing_2_list = pkl.load(f)
    with open("missing_1_list_7.pkl", "rb") as f:
        missing_1_list = pkl.load(f)
    with open("ridge_clf_list (3).pkl", "rb") as f:
        ridge_clf_list = pkl.load(f)
    with open("ridge_train_column_list (1).pkl", "rb") as f:
        ridge_train_column_list = pkl.load(f)
        
    imputer1 = pkl.load(open('imputer1_7.sav', 'rb'))
    
    present_list = pkl.load(open('median_present_list (1).pkl', 'rb'))
      
    df.replace([np.inf, -np.inf], np.nan,inplace=True)    
    
    df=df.drop(columns=missing_3_list).copy()    

    mean_imp_test_data  = imputer1.transform(df[missing_1_list])
    
    df.loc[:,missing_1_list]=mean_imp_test_data.copy()

    temp_list=[]
    for i in range(0,len(missing_2_list)):
        if df[missing_2_list[i]].isnull().bool():
            df["temp_"+str(missing_2_list[i])]=present_list[i]
        else:
            df["temp_"+str(missing_2_list[i])]=df[missing_2_list[i]]   
        temp_list.append("temp_"+missing_2_list[i]) 
    
    for i in range(0,len(missing_2_list)):
        train_columns=ridge_train_column_list[i]
        if df[missing_2_list[i]].isnull().bool():
            df[missing_2_list[i]] = ridge_clf_list[i].predict(df[train_columns]) 
    df.drop(columns=temp_list,inplace=True)
    return df
    

In [13]:
def calculate_cibil_score(df):
    cibil_test=df[['3365_LATE_PAYMENT_FLAG_MEAN','CRED_FLAG_LESS_30_MEAN','ABS_YEAR_CREDIT_MAX','UNSEC_LOAN_COUNT_SUM','SEC_LOAN_COUNT_SUM','AMT_REQ_CREDIT_BUREAU_WEEK']].copy()    
    
    scaler_cibil = pkl.load(open('scaler_cibil_7.sav', 'rb'))
    
    cibil_test_std = scaler_cibil.transform(cibil_test)
    
    cibil_test = pd.DataFrame(data = cibil_test_std,  
                      columns = ['3365_LATE_PAYMENT_FLAG_MEAN','CRED_FLAG_LESS_30_MEAN','ABS_YEAR_CREDIT_MAX','UNSEC_LOAN_COUNT_SUM','SEC_LOAN_COUNT_SUM','AMT_REQ_CREDIT_BUREAU_WEEK']) 
    
    num_test=(0.1*cibil_test['UNSEC_LOAN_COUNT_SUM'].copy()+0.1*cibil_test['SEC_LOAN_COUNT_SUM'].copy()+0.05*cibil_test['ABS_YEAR_CREDIT_MAX'].copy()+0.25*cibil_test['CRED_FLAG_LESS_30_MEAN'].copy())
    
    den_test=(0.30*cibil_test['3365_LATE_PAYMENT_FLAG_MEAN'].copy()+0.20*cibil_test['AMT_REQ_CREDIT_BUREAU_WEEK'].copy())+1
    
    df.loc[:,'CIBIL_SCORE']=(num_test.copy()/den_test.copy())
    
    df.loc[:,'CIBIL_SCORE']=df['CIBIL_SCORE'].fillna(0)
    
    return df

In [10]:
def main1(test_point):
    df=feature_engineering_on_app_train_test(test_point)
    with open("df_test.pkl", "rb") as f:
        df_past = pkl.load(f)
    with open("lgbm_clf_list_7.pkl", "rb") as f:
        lgbm_list = pkl.load(f)   
    with open("train_column7.pkl", "rb") as f:
        train_column = pkl.load(f)
    with open("scaler_7.sav", "rb") as f:
        scaler = pkl.load(f)
    df_past['SK_ID_CURR']=df_past.index
    test_point_past_data=df_past[df_past['SK_ID_CURR']==int(test_point['SK_ID_CURR'].values)]
    test_point_past_data.drop('SK_ID_CURR',axis=1,inplace=True)
    df=df.join(test_point_past_data,how='left', on='SK_ID_CURR')
    del df_past
    gc.collect()
    df=fill_the_missing_values(df)
    df=calculate_cibil_score(df)
    X=df[train_column]
    X_scaled=scaler.transform(X)
    test_pred_proba=0
    for j in range(0,len(lgbm_list)):
        test_pred_proba+=lgbm_list[j].predict_proba(X_scaled,num_iteration=lgbm_list[j].best_iteration_)[:,1]/10
    print('ID: ',df['SK_ID_CURR'].values,'Prediction: ',int(test_pred_proba>0.5),'Prediction Probablility: ',test_pred_proba)  

In [11]:
def main2(test_points,y_train):
    datatype_columns={}
    for x in test_points.columns:
        datatype_columns[x]=test_points[x].dtype
    y_predict=[]
    for i in range(0,len(test_points)):
        test_point=pd.DataFrame(test_points.values[i].reshape(1,-1),columns=list(test_points.iloc[i].index))
        test_point = test_point.astype(datatype_columns)
        df=feature_engineering_on_app_train_test(test_point)
        with open("df_test.pkl", "rb") as f:
            df_past = pkl.load(f)
        with open("lgbm_clf_list_7.pkl", "rb") as f:
            lgbm_list = pkl.load(f)   
        with open("train_column7.pkl", "rb") as f:
            train_column = pkl.load(f)
        with open("scaler_7.sav", "rb") as f:
            scaler = pkl.load(f)
        df_past['SK_ID_CURR']=df_past.index
        test_point_past_data=df_past[df_past['SK_ID_CURR']==int(test_point['SK_ID_CURR'].values)]
        test_point_past_data.drop('SK_ID_CURR',axis=1,inplace=True)
        df=df.join(test_point_past_data,how='left', on='SK_ID_CURR')
        del df_past
        gc.collect()
        df=fill_the_missing_values(df)
        df=calculate_cibil_score(df)
        X=df[train_column]
        X=scaler.transform(X)
        test_pred_proba=0
        for j in range(0,len(lgbm_list)):
             test_pred_proba+=lgbm_list[j].predict_proba(X,num_iteration=lgbm_list[j].best_iteration_)[:, 1]/10
        print('ID:',df['SK_ID_CURR'].values,'Prediction:',int(test_pred_proba>0.5),'Prediction Probablility:',test_pred_proba,'y_actual: ',y_train.iloc[i])  
        y_predict.append(test_pred_proba)
    print('ROC-AUC score',roc_auc_score(y_train,y_predict))

In [4]:
app_train=pd.read_csv('application_train.csv')
app_test=pd.read_csv('application_test.csv')

In [363]:
app_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [364]:
test_point=app_test.sample(1)

In [368]:
main1(test_point)

ID:  [100001] Prediction:  0 Prediction Probablility:  [0.034699]


In [5]:
train_points=app_train.sample(80)
train_points
y=train_points['TARGET']
X_train=train_points.drop('TARGET',axis=1)

In [373]:
main2(X_train,y)

ID: [340121] Prediction: 0 Prediction Probablility: [0.17251587] y_actual:  0
ID: [238870] Prediction: 0 Prediction Probablility: [0.07623851] y_actual:  0
ID: [445290] Prediction: 0 Prediction Probablility: [0.01293887] y_actual:  0
ID: [436999] Prediction: 0 Prediction Probablility: [0.02155605] y_actual:  0
ID: [362325] Prediction: 0 Prediction Probablility: [0.01248308] y_actual:  0
ID: [377861] Prediction: 0 Prediction Probablility: [0.02249652] y_actual:  0
ID: [454060] Prediction: 0 Prediction Probablility: [0.13889779] y_actual:  1
ID: [294906] Prediction: 0 Prediction Probablility: [0.02564005] y_actual:  0
ID: [319469] Prediction: 0 Prediction Probablility: [0.01655263] y_actual:  0
ID: [349319] Prediction: 0 Prediction Probablility: [0.05786691] y_actual:  0
ID: [101927] Prediction: 0 Prediction Probablility: [0.0133861] y_actual:  0
ID: [260511] Prediction: 0 Prediction Probablility: [0.05819658] y_actual:  0
ID: [102696] Prediction: 0 Prediction Probablility: [0.10875774] 